# Overview

This notebook uses the 'TextFeaturesGenerator' class (from text_features) to convert textual data into qunatitaive data. 

For now, it creates a bag-of-words representation and a tf-idf representation. We will also add SVD/PCA of these matrices and a Word2Vec representation in the next few days.

Will update the TextFeaturesGenerator class on an ongoing basis and update the usage here.

In [1]:
from text_features import TextFeaturesGenerator
import pandas as pd

In [17]:
tweets = pd.read_csv("trump_archive_db.csv",encoding = 'unicode_escape',names=['text'])

In [18]:
tweets = tweets.dropna()

In [19]:
tweets.head()

,text
0,DEMOCRATS WANT TO STEAL THE ELECTION! #KAG2020...
1,Mississippi there is a VERY important election...
2,....He loves our Military and supports our Vet...
3,LOOK AT THIS PHOTOGRAPH! https://t.co/QQYTqG4K...
4,Schiff House Intel Chairman Got Early Account...


In [22]:
feature_generator = TextFeaturesGenerator(tweets.text)

In [23]:
bow_mat = feature_generator.get_bow_matrix()

In [24]:
bow_mat.shape

(27262, 31079)

In [25]:
tfidf_mat = feature_generator.get_tfidf_matrix()
tfidf_mat.shape

(27262, 31079)

In [26]:
feature_generator.save_matrices()